In [66]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [117]:
tripdata_jan21 = pd.read_parquet('fhv_tripdata_2021-01.parquet')
tripdata_feb21 = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [118]:
tripdata_jan21.shape[0]

1154112

## Q2. Computing duration

In [119]:
tripdata_jan21

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


In [120]:
tripdata_jan21['duration'] = tripdata_jan21.apply(lambda x: pd.to_datetime(x.dropOff_datetime, infer_datetime_format=True) - pd.to_datetime(x.pickup_datetime, infer_datetime_format=True), axis = 1)
tripdata_jan21['duration'] = tripdata_jan21['duration'].astype('timedelta64[s]')/60

In [121]:
tripdata_jan21['duration'].mean()

19.1672240937939

In [122]:
tripdata_jan21['duration'].describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [123]:
tripdata_jan21_no_outliers = tripdata_jan21.query('(duration >= 1) & (duration <= 60)')

print('We removed ' + str(tripdata_jan21.shape[0] - tripdata_jan21_no_outliers.shape[0]) + ' records, by filtering for duration between 1 and 60 mins inclusive.')

We removed 44286 records, by filtering for duration between 1 and 60 mins inclusive.


## Q3. Missing values

In [124]:
na_columns = ['PUlocationID','DOlocationID']

for col in na_columns:
    tripdata_jan21_no_outliers[col].fillna(-1, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [125]:
missing_value_count = tripdata_jan21_no_outliers.query('PUlocationID == -1').shape[0]
missing_value_fraction = missing_value_count/ tripdata_jan21_no_outliers.shape[0] * 100
missing_value_fraction

83.52732770722618

## Q4. One-hot encoding

One advantage of using DictVectorizer instead of get_dummies is that it supports sparse matrix output. Thus, if we have a large number of features, DictVectorizer will be a good choice:

- Reference: https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [126]:
train_dicts = tripdata_jan21_no_outliers[na_columns].astype(str).to_dict(orient='records')   #convert df to dict

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [127]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [128]:
X_train.shape[1]

525

525 columns, each PUlocationID or DOlocationID becomes a column/feature

## Q5. Training a model

In [129]:
y_train = tripdata_jan21_no_outliers['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107210744

## Q6. Evaluating the model

In [130]:
#Repeat same preprocessing for validation set
tripdata_feb21['duration'] = tripdata_feb21.apply(lambda x: pd.to_datetime(x.dropOff_datetime, infer_datetime_format=True) - pd.to_datetime(x.pickup_datetime, infer_datetime_format=True), axis = 1)
tripdata_feb21['duration'] = tripdata_feb21['duration'].astype('timedelta64[s]')/60

tripdata_feb21_no_outliers = tripdata_feb21.query('(duration >= 1) & (duration <= 60)')

for col in na_columns:
    tripdata_feb21_no_outliers[col].fillna(-1, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [146]:
val_dicts = tripdata_feb21_no_outliers[na_columns].astype(str).to_dict(orient='records')

X_val = dv.transform(val_dicts)           #Apply previous dictvectorizer on validation set
y_val = tripdata_feb21_no_outliers['duration'].values

y_pred_val = lr.predict(X_val)

mean_squared_error(y_val, y_pred_val, squared=False)

11.014283196111764